# Imports

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt 
from collections import Counter
from tqdm import tqdm
import pickle

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

from tslearn.clustering import TimeSeriesKMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

# Set Tensorflow 

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Set random seed for reproducibility

In [ ]:
# np.random.seed(1234)
# tf.random.set_seed(1234)

# Loading Data

In [ ]:
with open("options.txt", 'r') as f:
    options = f.readlines()
    options = {option.split("=")[0]: option.split("=")[1].strip() for option in options}
print(options)

In [ ]:
RUG = pd.read_pickle(options['RUG_no_outliers'])

# Preparing and Transforming Data

In [ ]:
RUG.interpolate(method='linear', inplace=True, limit=20)

In [ ]:
def get_data(col_name):
    # copy data to avoid changing the original data
    df = RUG[col_name].copy()
    
    # group data by day
    groups = df.groupby(pd.Grouper(freq='D'))

    # get the calender date of the groups
    days = list(groups.first().index.strftime('%Y:%m:%d'))

    # create a list of dataframes for each day
    gro = [groups.get_group(x).reset_index(drop=True) for x in groups.groups]

    # create a single dataframe with all days as columns
    temp = pd.concat(gro, axis=1, keys=days)

    # set index to hours and minutes
    temp.index = pd.date_range("00:00", "23:59", freq="1min").strftime('%H:%M')

    # drop all columns of temp dataframe which contain nan values
    temp.dropna(axis=1, how='any', inplace=True)

    # reduce data to every 10 minutes
    temp = temp[::10]
    
    # return transformed data 
    return temp

In [ ]:
def scale_data(data):
    # copy data to avoid changing the original data
    data_copy = data.copy()

    # create train and test set based on train_percentage
    train_percentage = 0.8
    train_size = int(len(data_copy.columns) * train_percentage)

    train = data_copy.iloc[:, :train_size]
    test = data_copy.iloc[:, train_size:]

    # create scaler object
    scaler = MinMaxScaler(feature_range=(0, 1))

    # fit and transform scaler to train data
    scaled_list_train = [train[col] for col in train]
    scaled_list_train = scaler.fit_transform(scaled_list_train)
    
    # transform test data 
    scaled_list_test = [test[col] for col in test]
    scaled_list_test = scaler.transform(scaled_list_test)

    return scaler, scaled_list_train, scaled_list_test

# Principal Component Analysis

In [ ]:
def create_pca(data):
    # copy data to avoid changing the original data
    data_copy = data.copy()
    
    pca = PCA(n_components=0.85, svd_solver='full')
    
    # Fit and transform data
    pca_features = pca.fit_transform(data_copy)

    return pca_features

In [ ]:
def create_kmeans(pca_data, scaled_train, scaled_test, clusters=4):
    # copy data to avoid changing the original data
    temp_pca_data = pca_data.copy()
    temp_scaled_train = scaled_train.copy()
    temp_scaled_test = scaled_test.copy()

    # fit kmeans to pca data
    kmeans_pca = TimeSeriesKMeans(n_clusters=clusters, metric="dtw", n_jobs=-1).fit(temp_pca_data)
    
    # extract and predict cluster labels
    train_pca_features = kmeans_pca.labels_
    test_pca_features = kmeans_pca.predict(temp_scaled_test)

    return train_pca_features, test_pca_features

# Train different lstm models

In [ ]:
def func(train1, test1, scaler, look_back=3):
    # copy data to avoid changing the original data
    training, testing = train1.copy(), test1.copy()

    # set amount of previous timesteps to use as input
    look_back = 3
    
    # convert a datapoint (time series) to vertical format and 
    # add column for previous known values using look_back
    def create_dataset(dataset, look_back=3):
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i+look_back), 0]
            dataX.append(a)
            dataY.append(dataset[i + look_back, 0])
        return np.array(dataX), np.array(dataY)

    # set callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=2, min_lr=0.001, verbose=2)

    # create and fit the LSTM network
    model = Sequential()
    model.add(LSTM(4, input_shape=(1, look_back)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

    # check if there are more than one training time series assigned to the cluster
    if training.ndim > 1:
        # train model on each time series
        for train_it in tqdm(training): 
            # transform time series into data which can be used as input for the model
            train_it = train_it.reshape(-1, 1)
            trainX, trainY = create_dataset(train_it, look_back)
            trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))

            # fit model, model data is not reset between running of time series
            model.fit(trainX, trainY, epochs=50, verbose=0, callbacks=[early_stopping, reduce_lr])
    else:
        # if there is only one time series, dont iterate over it
        # same steps as above
        train_it = training
        train_it = train_it.reshape(-1, 1)
        
        trainX, trainY = create_dataset(train_it, look_back)
        trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))

        model.fit(trainX, trainY, epochs=50, verbose=0, callbacks=[early_stopping, reduce_lr])

    # create lists to store metrics
    rmse_train = []
    rmse_test = []

    mae_train = []
    mae_test = []

    mape_train = []
    mape_test = []

    # check if there are more than one training time series assigned to the cluster
    if training.ndim > 1:
        for train_it in training:
            # transform time series into data which can be used as input for the model
            train_it = train_it.reshape(-1, 1)
            trainX, trainY = create_dataset(train_it, look_back)
            trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
            
            # predict values for train data
            trainPredict = model.predict(trainX, verbose=0)
            
            # invert predictions and actual values to original scale
            trainPredict = np.repeat(trainPredict, train1.shape[1], axis=-1)
            trainPredict = scaler.inverse_transform(trainPredict)[:,0]
            
            trainY = np.repeat(trainY.reshape(-1, 1), train1.shape[1], axis=-1)
            trainY = scaler.inverse_transform(trainY)[:,0]
            
            # calculate and store metrics based on original values
            rmse_train.append(np.sqrt(mean_squared_error(trainY, trainPredict)))
            mae_train.append(tf.keras.metrics.mean_absolute_error(trainY, trainPredict).numpy())
            mape_train.append(tf.keras.metrics.mean_absolute_percentage_error(trainY, trainPredict).numpy())
    # if there is only one training time series, dont iterate over it
    # same steps as above
    else:
        train_it = training
        train_it = train_it.reshape(-1, 1)

        trainX, trainY = create_dataset(train_it, look_back)

        trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
        
        trainPredict = model.predict(trainX, verbose=0)
        trainPredict = np.repeat(trainPredict, train1.shape[1], axis=-1)
        trainPredict = scaler.inverse_transform(trainPredict)[:,0]
        
        trainY = np.repeat(trainY.reshape(-1, 1), train1.shape[1], axis=-1)
        trainY = scaler.inverse_transform(trainY)[:,0]
        
        rmse_train.append(np.sqrt(mean_squared_error(trainY, trainPredict)))
        mae_train.append(tf.keras.metrics.mean_absolute_error(trainY, trainPredict).numpy())
        mape_train.append(tf.keras.metrics.mean_absolute_percentage_error(trainY, trainPredict).numpy())

    # same as for training data
    if testing.ndim > 1:
        for test_it in testing:   
            try:
                test_it = test_it.reshape(-1, 1) 
                testX, testY = create_dataset(test_it, look_back)
                testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

                testPredict = model.predict(testX, verbose=0)
                testPredict = np.repeat(testPredict, test1.shape[1], axis=-1)
                testPredict = scaler.inverse_transform(testPredict)[:,0]

                testY = np.repeat(testY.reshape(-1, 1), test1.shape[1], axis=-1)
                testY = scaler.inverse_transform(testY)[:,0]

                rmse_test.append(np.sqrt(mean_squared_error(testY, testPredict)))
                mae_test.append(tf.keras.metrics.mean_absolute_error(testY, testPredict).numpy())
                mape_test.append(tf.keras.metrics.mean_absolute_percentage_error(testY, testPredict).numpy())
            except:
                print("exception occured")
                rmse_train.append(-1)
                mae_train.append(-1)
                mape_train.append(-1)
    # if there is only one testing time series, dont iterate over it
    # same steps as above
    else:
        try:
            test_it = testing
            test_it = test_it.reshape(-1, 1) 
            testX, testY = create_dataset(test_it, look_back)

            testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

            testPredict = model.predict(testX, verbose=0)
            testPredict = np.repeat(testPredict, test1.shape[1], axis=-1)
            testPredict = scaler.inverse_transform(testPredict)[:,0]

            testY = np.repeat(testY.reshape(-1, 1), test1.shape[1], axis=-1)
            testY = scaler.inverse_transform(testY)[:,0]

            rmse_test.append(np.sqrt(mean_squared_error(testY, testPredict)))
            mae_test.append(tf.keras.metrics.mean_absolute_error(testY, testPredict).numpy())
            mape_test.append(tf.keras.metrics.mean_absolute_percentage_error(testY, testPredict).numpy())
        except:
            print("exception occured")
            rmse_test.append(-1)
            mae_test.append(-1)
            mape_test.append(-1)

    return (rmse_train, rmse_test, mae_train, mae_test, mape_train, mape_test)

# Num of clusters per column

based on elbow method and silhouette score

In [ ]:
clusters = [4, 4, 3, 3, 4, 4, 4, 3, 3, 4, 3, 4, 4]

In [ ]:
complete_results = []
# runs the needed functions for each location
for location, clust_n in zip(RUG.columns, clusters):
    print(location)
    data = get_data(location)

    scaler, scaled_list_train, scaled_list_test = scale_data(data)
    
    pca_features = create_pca(scaled_list_train)

    train_pca_features, test_pca_features = create_kmeans(pca_features, scaled_list_train, scaled_list_test, clust_n)
    # print(Counter(train_pca_features), Counter(test_pca_features))

    # for each unique assigned cluster label 
    for cluster in [*Counter(train_pca_features)]:
        # select the data points that belong to that cluster
        cluster_train = scaled_list_train[np.where(train_pca_features == cluster)]
        cluster_test = scaled_list_test[np.where(test_pca_features == cluster)]

        # run the model based on the cluster
        reply = func(cluster_train, cluster_test, scaler)

        # add results to the complete results list
        complete_results.append([location, [cluster, [np.mean(reply[0]), np.mean(reply[1]), np.mean(reply[2]), np.mean(reply[3]), np.mean(reply[4]), np.mean(reply[5])]]])

# save results
with open (r"results_non_clustering.txt", 'wb') as f:
    pickle.dump(complete_results, f)